In [1]:
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#import sklearn as skl
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator


def lab_encode(dataset):#Ho una sola colonna! 
    new_le = preproc.LabelEncoder()
    dataset['dx'] = new_le.fit_transform(dataset['dx']) 
    #  label_encoders[col] = new_le 
    return

image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_path = 'HAM10000_metadata.csv'
dataset = pd.read_csv(csv_path, encoding = "ISO-8859-1")
lab_encode(dataset)



    # apro il csv
    # encoding del dx
    # prendo solo le colonne name e dx
    # per ogni file:
    # os.replace("path/to/current/file.foo", "path/to/new/destination/for/file.foo")

Using TensorFlow backend.


In [ ]:
########### gioco di cartelle
for index in range(len(dataset)):
    img=dataset.iloc[index]["image_id"]
    source = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1\\'+ img +'.jpg'
    dx =str(dataset.iloc[index]["dx"])
    dest = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1\\'+ dx + '\\' + img + '.jpg'
    os.replace(dest, source)

In [ ]:
import shutil

# note that we are not augmenting class 'nv'
class_list = ['0','1','2','3','4','6']

# We are creating temporary directories here because we delete these directories later
# create a base dir
aug_dir = '..\\aug_dir'
#os.mkdir(aug_dir)
# create a dir within the base dir to store images of the same class
img_dir = os.path.join(aug_dir, 'img_dir')
#os.mkdir(img_dir2)

for item in class_list:
    class_dir = os.path.join(img_dir2, item)
    os.mkdir(class_dir)

    # Choose a class
    img_class = item

    # list all images in that directory
    img_list = os.listdir('..\\skin-cancer-mnist-ham10000\\HAM10000_images_part_1\\' + img_class) # qui le immagini sono già nelle proprie cartelle

    # Copy images from the class train dir to the img_dir e.g. class 'mel'
    for fname in img_list:
            # source path to image
            src = os.path.join('..\\skin-cancer-mnist-ham10000\\HAM10000_images_part_1\\' + img_class, fname)
            # destination path to image
            dst = os.path.join(class_dir, fname) #####
            #print(dst)
            # copy the image from the source to the destination
            ####################shutil.copyfile(src, dst)


    # point to a dir containing the images and not to the images themselves
    path = img_dir
    #save_path = img_dir + '\\' + img_class
    save_path = img_dir  # la cartella label esiste già

    # Create a data generator
    datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        #brightness_range=(0.9,1.1),
        fill_mode='nearest')

    batch_size = 50

    aug_datagen = datagen.flow_from_directory(path,
                                           save_to_dir=save_path,
                                           save_format='jpg',
                                                    target_size=(224,224),
                                                    batch_size=batch_size)



    # Generate the augmented images and add them to the training folders
    
    ###########
    
    num_aug_images_wanted = 6000 # total number of images we want to have in each class
    
    ###########
    
    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))

    # run the generator and create about 6000 augmented images
    for i in range(0,num_batches):

        imgs, labels = next(aug_datagen)
        
    # delete temporary directory with the raw image files
    #shutil.rmtree('aug_dir')

In [ ]:
import shutil
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# note that we are not augmenting class 'nv'
class_list = ['0','1','2','3','4','6']

# We are creating temporary directories here because we delete these directories later
# create a base dir
aug_dir = '..\\aug_dir'
#os.mkdir(aug_dir)
# create a dir within the base dir to store images of the same class
img_dir = os.path.join(aug_dir, 'img_dir')
#os.mkdir(img_dir2)
class_dir = os.path.join(img_dir, '0')
#os.mkdir(class_dir)

# Choose a class
img_class = '0'

# list all images in that directory
img_list = os.listdir('..\\skin-cancer-mnist-ham10000\\HAM10000_images_part_1\\' + img_class) # qui le immagini sono già nelle proprie cartelle

print('#img: '+ str(len(img_list)))

# Copy images from the class train dir to the img_dir e.g. class 'mel'
for fname in img_list:
        # source path to image
        src = os.path.join('..\\skin-cancer-mnist-ham10000\\HAM10000_images_part_1\\' + img_class, fname)
        # destination path to image
        dst = os.path.join(class_dir, fname) #####
        #print('src: ' + src + '\t\t dst: ' + dst)
        # copy the image from the source to the destination
        ####################shutil.copyfile(src, dst)


# point to a dir containing the images and not to the images themselves
path = img_dir+'\\'
#save_path = img_dir + '\\' + img_class
save_path = img_dir  # la cartella label esiste già

# Create a data generator
datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    #brightness_range=(0.9,1.1),
    fill_mode='nearest')

img = Image.open('..\\skin-cancer-mnist-ham10000\\HAM10000_images_part_1\\0\\ISIC_0024329.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=class_dir, save_prefix='cat', save_format='jpg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

# run the generator and create about 6000 augmented images
#for i in range(0,num_batches):
#
#    imgs, labels = next(aug_datagen)
    
# delete temporary directory with the raw image files
#shutil.rmtree('aug_dir')


In [ ]:
k = datagen.flow(x, batch_size=5, save_to_dir=class_dir, save_prefix='dog', save_format='jpg')

In [ ]:
import gc
gc.collect()

In [10]:
############# rescaling immagini
src_path = "../../skin-cancer-mnist-ham10000/HAM10000_images_part_1"
dst_path = "../../Untitled Folder"
size1=(200, 150)
size2=(300, 225)

    
for fname in os.listdir(src_path):
    img_rsz = Image.open(os.path.join(src_path, fname)).resize(sizeX)
    img_rsz.save(os.path.join(dst_path, fname))